In [1]:
import time
import tensorflow as tf
import psutil

class PowerMonitor:
    def __init__(self):
        self.gpu_available = tf.config.list_physical_devices('GPU')
        
        # Hardware power specifications (adjust these values for your system)
        self.cpu_tdp = 65    # Typical TDP for desktop CPUs in watts
        self.gpu_tdp = 250   # Typical TDP for desktop GPUs in watts
        
    def get_stats(self):
        """Get system stats with power estimation"""
        stats = {
            'timestamp': time.time(),
            'cpu_%': psutil.cpu_percent(interval=0.1),
            'ram_mb': psutil.virtual_memory().used / (1024**2),
            'gpu_mem_mb': 0,
            'power_w': self.cpu_tdp * (psutil.cpu_percent()/100) * 0.85  # Base CPU power
        }
        
        if self.gpu_available:
            try:
                # TensorFlow GPU memory monitoring
                mem_info = tf.config.experimental.get_memory_info('GPU:0')
                stats.update({
                    'gpu_mem_mb': mem_info['current'] / (1024**2),
                    'power_w': self.cpu_tdp * (psutil.cpu_percent()/100) * 0.85 + 
                              self.gpu_tdp * 0.5 * 0.75  # Add GPU power estimate
                })
            except:
                pass
                
        return stats

# Initialize monitor
monitor = PowerMonitor()

# Celeb

In [2]:

import h5py

# Open the HDF5 file for reading
#with h5py.File("D:\\thesis\dataset\deepfake dataset\images_celeb_224R_processed.h5", "r") as h5f:
with h5py.File("D:\\thesis\dataset\Celeb-Df-v2\images_celeb_balanced_224R_processed.h5", "r") as h5f:
    # Load HOG features
    real_frames_array1 = h5f["ori_actor"][:]
    fake_frames_array1 = h5f["ori_youtube"][:]
import numpy as np
import cv2

# Function to resize images from (224, 224) to (160, 160)
def resize_images(image_array, target_size=(160, 160)):
    resized_images = np.array([cv2.resize(img, target_size) for img in image_array])
    return resized_images


# Resize the images
real_frames_array1 = resize_images(real_frames_array1, target_size=(160, 160))
fake_frames_array1 = resize_images(fake_frames_array1, target_size=(160, 160))

# Checking the new shapes
real_frames_array1.shape, fake_frames_array1.shape



((9870, 160, 160, 3), (11274, 160, 160, 3))

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


# Split the data into train (70%) and test (30%)
X_train_real, X_test_real = train_test_split(real_frames_array1, test_size=0.3, random_state=42)
X_train_fake, X_test_fake = train_test_split(fake_frames_array1, test_size=0.3, random_state=42)


# Output the shape to confirm the split dimensions
print("Training data real shape:", X_train_real.shape)
print("Testing real data shape:", X_test_real.shape)
print("Training fake data shape:", X_train_fake.shape)
print("Testing  fake data shape:", X_test_fake.shape)
########################################################################################################################################
#######################################divide into 60,10 train and val
#########################################################################################################################################
def extract_validation(train_data):
    """
    Extract every 10th sample from the training data and store it in a validation set.

    Parameters:
        train_data (list or np.array): The training dataset.

    Returns:
        tuple: Updated training dataset and validation dataset.
    """
    # Select every 10th sample for the validation set
    validation_data = train_data[::10]

    # Remove the selected samples from the training dataset
    updated_train_data = [train_data[i] for i in range(len(train_data)) if i % 10 != 0]

    return np.array(updated_train_data), np.array(validation_data)


# Perform the operation for each dataset
train_hog_real, val_hog_real = extract_validation(X_train_real)
train_hog_fake, val_hog_fake = extract_validation(X_train_fake)

# Print results for verification
print(f"train_hog_real: {len(train_hog_real)} images, val_hog_real: {len(val_hog_real)} images")
print(f"train_hog_fake: {len(train_hog_fake)} images, val_hog_fake: {len(val_hog_fake)} images")


############################################################################################################################################################
#################################################concatenate the labels 0,1 real and fake
#############################################################################################################################################################


train_labels_real = np.zeros(len(train_hog_real), dtype=int)
train_labels_fake = np.ones(len(train_hog_fake), dtype=int)

# Concatenate all training datasets into a single `train` variable
train = np.concatenate([train_hog_real, train_hog_fake], axis=0)
train_labels=np.concatenate([train_labels_real, train_labels_fake], axis=0)



test_labels_real = np.zeros(len(X_test_real), dtype=int)
test_labels_fake = np.ones(len(X_train_fake), dtype=int)

# Concatenate all testing datasets into a single `test` variable
test = np.concatenate([X_test_real, X_train_fake], axis=0)
test_labels = np.concatenate([test_labels_real, test_labels_fake], axis=0)



val_labels_real = np.zeros(len(val_hog_real), dtype=int)
val_labels_fake = np.ones(len(val_hog_fake), dtype=int)

# Concatenate all validation datasets into a single `val` variable
val = np.concatenate([val_hog_real, val_hog_fake], axis=0)
val_labels = np.concatenate([val_labels_real, val_labels_fake], axis=0)

# Print the results for verification
print(f"Total train: {len(train)} images")
print(f"Total test: {len(test)} images")
print(f"Total val: {len(val)} images")


# Print results for verification
print(f"Train Labels: {len(train_labels)} ")
print(f"Test Labels: {len(test_labels)} ")
print(f"Val Labels: {len(val_labels)} ")



Training data real shape: (6909, 160, 160, 3)
Testing real data shape: (2961, 160, 160, 3)
Training fake data shape: (7891, 160, 160, 3)
Testing  fake data shape: (3383, 160, 160, 3)
train_hog_real: 6218 images, val_hog_real: 691 images
train_hog_fake: 7101 images, val_hog_fake: 790 images
Total train: 13319 images
Total test: 10852 images
Total val: 1481 images
Train Labels: 13319 
Test Labels: 10852 
Val Labels: 1481 


In [4]:
print("=== DATA LOADING ===")
start = monitor.get_stats()

=== DATA LOADING ===


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, callbacks
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Configuration
INPUT_SHAPE = (160, 160, 3)
NUM_CLASSES = 1  # Binary classification (real=0, fake=1)
BATCH_SIZE = 32
EPOCHS = 100

# Load pre-trained InceptionV3 (excluding top layers)
base_model = InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=INPUT_SHAPE
)

# Freeze initial layers (optional)
for layer in base_model.layers[:100]:
    layer.trainable = False

# Custom Head for Deepfake Detection
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(NUM_CLASSES, activation='sigmoid')(x)

# Final Model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile with class weights (adjust if dataset is imbalanced)
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)


# Model Summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 79, 79, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 79, 79, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [6]:
# Train
history = model.fit(
    train, train_labels,
    validation_data=(val, val_labels),
    epochs=100,
    batch_size=32
)

Epoch 1/100
417/417 [==============================] - 38s 66ms/step - loss: 0.6351 - accuracy: 0.6425 - auc: 0.6927 - val_loss: 0.6737 - val_accuracy: 0.6617 - val_auc: 0.7195
Epoch 2/100
417/417 [==============================] - 27s 64ms/step - loss: 0.5182 - accuracy: 0.7408 - auc: 0.8192 - val_loss: 0.5273 - val_accuracy: 0.7292 - val_auc: 0.8302
Epoch 3/100
417/417 [==============================] - 26s 63ms/step - loss: 0.4299 - accuracy: 0.7983 - auc: 0.8821 - val_loss: 0.4737 - val_accuracy: 0.7650 - val_auc: 0.8649
Epoch 4/100
417/417 [==============================] - 27s 64ms/step - loss: 0.3601 - accuracy: 0.8378 - auc: 0.9193 - val_loss: 0.4900 - val_accuracy: 0.7738 - val_auc: 0.8684
Epoch 5/100
417/417 [==============================] - 26s 62ms/step - loss: 0.3153 - accuracy: 0.8639 - auc: 0.9386 - val_loss: 0.4218 - val_accuracy: 0.8136 - val_auc: 0.9040
Epoch 6/100
417/417 [==============================] - 26s 62ms/step - loss: 0.2722 - accuracy: 0.8842 - auc: 0.955

In [6]:
# Train
history = model.fit(
    train, train_labels,
    validation_data=(val, val_labels),
    epochs=100,
    batch_size=32
)

Epoch 1/100
417/417 [==============================] - 44s 80ms/step - loss: 0.6417 - accuracy: 0.6273 - auc: 0.6796 - val_loss: 0.5922 - val_accuracy: 0.6941 - val_auc: 0.7631
Epoch 2/100
417/417 [==============================] - 32s 77ms/step - loss: 0.5235 - accuracy: 0.7366 - auc: 0.8149 - val_loss: 0.5189 - val_accuracy: 0.7448 - val_auc: 0.8312
Epoch 3/100
417/417 [==============================] - 32s 77ms/step - loss: 0.4342 - accuracy: 0.7941 - auc: 0.8792 - val_loss: 0.4489 - val_accuracy: 0.7833 - val_auc: 0.8763
Epoch 4/100
417/417 [==============================] - 31s 75ms/step - loss: 0.3613 - accuracy: 0.8362 - auc: 0.9180 - val_loss: 0.4346 - val_accuracy: 0.7893 - val_auc: 0.8837
Epoch 5/100
417/417 [==============================] - 31s 75ms/step - loss: 0.3094 - accuracy: 0.8647 - auc: 0.9409 - val_loss: 0.3943 - val_accuracy: 0.8373 - val_auc: 0.9095
Epoch 6/100
417/417 [==============================] - 32s 76ms/step - loss: 0.2622 - accuracy: 0.8893 - auc: 0.957

In [8]:
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts


=== RESOURCE USAGE ===
CPU Usage: 15.2%
RAM Used: -3787.8 MB
Time Usage: 6240.2 s
GPU Memory Used: 1069.1 MB
Power Consumption: 149W


In [7]:
# Evaluate on test set
test_loss, test_acc, test_auc = model.evaluate(test, test_labels)
print(f'\nTest Accuracy of Celeb-DF on InceptionV3: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred_binary))

340/340 [==============================] - 17s 49ms/step - loss: 0.1645 - accuracy: 0.9563 - auc: 0.9765

Test Accuracy of Celeb-DF on InceptionV3: 95.63%
340/340 [==============================] - 27s 51ms/step
              precision    recall  f1-score   support

           0       0.96      0.87      0.92      2961
           1       0.95      0.99      0.97      7891

    accuracy                           0.96     10852
   macro avg       0.96      0.93      0.94     10852
weighted avg       0.96      0.96      0.96     10852



In [9]:
# Evaluate on test set
test_loss, test_acc, test_auc = model.evaluate(test, test_labels)
print(f'\nTest Accuracy of Celeb-DF on InceptionV3: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred_binary))

340/340 [==============================] - 11s 32ms/step - loss: 0.1845 - accuracy: 0.9551 - auc: 0.9733

Test Accuracy of Celeb-DF on InceptionV3: 95.51%
340/340 [==============================] - 11s 29ms/step
              precision    recall  f1-score   support

           0       0.97      0.86      0.91      2961
           1       0.95      0.99      0.97      7891

    accuracy                           0.96     10852
   macro avg       0.96      0.93      0.94     10852
weighted avg       0.96      0.96      0.95     10852



In [10]:

import tensorflow as tf

# After training the model
model.save('InceptionV3_160_celeb.h5')  # Saves the entire model to a file


In [11]:
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts


=== RESOURCE USAGE ===
CPU Usage: 7.4%
RAM Used: -1394.8 MB
Time Usage: 3346.4 s
GPU Memory Used: 1069.0 MB
Power Consumption: 93W


In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('InceptionV3_160_celeb.h5')
test_loss, test_acc = model.evaluate(test, test_labels)
print(f'Test Accuracy of Celeb-df on InceptionV3: {test_acc * 100:.2f}%')
# You can now use the model for testing or inference


340/340 [==============================] - 2s 4ms/step - loss: 0.1922 - accuracy: 0.9294
Test Accuracy of Celeb-df on Meso NET: 92.94%


In [4]:
from tensorflow.keras.models import load_model
import tensorflow as tf
print("=== DATA LOADING ===")
start = monitor.get_stats()
# Load the saved model
model = tf.keras.models.load_model('InceptionV3_160_celeb.h5')
# Evaluate on test set
test_loss, test_acc,test_auc = model.evaluate(test, test_labels)
print(f'Test Accuracy of Celeb-df on InceptionV3: {test_acc * 100:.2f}%')
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts

=== DATA LOADING ===
340/340 [==============================] - 17s 35ms/step - loss: 0.1845 - accuracy: 0.9551 - auc: 0.9733
Test Accuracy of Celeb-df on InceptionV3: 95.51%

=== RESOURCE USAGE ===
CPU Usage: 7.0%
RAM Used: 3160.9 MB
Time Usage: 23.6 s
GPU Memory Used: 1081.9 MB
Power Consumption: 93W


In [12]:
test_loss, test_acc, test_auc= model.evaluate(test_hog, test_labels)
print(f'\nTest Accuracy of DFC on Xceptionnet from Celebdf: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test_hog)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred_binary))

94/94 [==============================] - 3s 33ms/step - loss: 2.6619 - accuracy: 0.5080 - auc: 0.5208

Test Accuracy of DFC on Xceptionnet from Celebdf: 50.80%
94/94 [==============================] - 3s 32ms/step
              precision    recall  f1-score   support

           0       0.50      0.93      0.65      1500
           1       0.55      0.09      0.15      1500

    accuracy                           0.51      3000
   macro avg       0.53      0.51      0.40      3000
weighted avg       0.53      0.51      0.40      3000



In [6]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('InceptionV3_160_celeb.h5')
# Evaluate on test set
test_loss, test_acc, test_auc = model.evaluate(test_ff, test_labels_ff)
print(f'\nTest Accuracy of FF++ on Inceptionnet from Celebdf: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test_ff)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels_ff, y_pred_binary))

169/169 [==============================] - 46s 266ms/step - loss: 4.6027 - accuracy: 0.3776 - auc: 0.5703

Test Accuracy of FF++ on Inceptionnet from Celebdf: 37.76%
169/169 [==============================] - 45s 261ms/step
              precision    recall  f1-score   support

           0       0.29      0.92      0.45      1468
           1       0.85      0.17      0.29      3911

    accuracy                           0.38      5379
   macro avg       0.57      0.55      0.37      5379
weighted avg       0.70      0.38      0.33      5379



# DFC

In [2]:
import h5py
import numpy as np
# Open the HDF5 file in read mode
with h5py.File('D://thesis//dataset//deepfake dataset//resized_images.h5', 'r') as h5f:
    # Access each dataset
    celeb = np.array(h5f['celeb'])
    ffhq = np.array(h5f['ffhq'])
    gdwct = np.array(h5f['gdwct'])
    attgan = np.array(h5f['attgan'])
    stargan = np.array(h5f['stargan'])
    stylegan2 = np.array(h5f['stylegan2'])
    stylegan = np.array(h5f['stylegan'])

# Now, 'celeb', 'ffhq', etc., are NumPy arrays containing your datasets
print(f"celeb shape: {celeb.shape}, dtype: {celeb.dtype}")
print(f"ffhq shape: {ffhq.shape}, dtype: {ffhq.dtype}")
print(f"ffhq shape: {gdwct.shape}, dtype: {gdwct.dtype}")
print(f"ffhq shape: {attgan.shape}, dtype: {attgan.dtype}")
print(f"ffhq shape: {stargan.shape}, dtype: {stargan.dtype}")
print(f"ffhq shape: {stylegan2.shape}, dtype: {stylegan2.dtype}")
print(f"ffhq shape: {stylegan.shape}, dtype: {stylegan.dtype}")
# Repeat for other datasets as needed
import cv2
# Function to resize images from (224, 224) to (160, 160)
def resize_images(image_array, target_size=(160, 160)):
    resized_images = np.array([cv2.resize(img, target_size) for img in image_array])
    return resized_images

celeb = resize_images(celeb, target_size=(160, 160))
ffhq = resize_images(ffhq, target_size=(160, 160))
gdwct = resize_images(gdwct, target_size=(160, 160))
attgan = resize_images(attgan, target_size=(160, 160))
stargan = resize_images(stargan, target_size=(160, 160))
stylegan = resize_images(stargan, target_size=(160, 160))
stylegan2 = resize_images(stylegan2, target_size=(160, 160))
import random
# Randomly select 2500 distinct images
random_indices = random.sample(range(len(celeb)), 2500)  # Get 2500 random indices
celeb = celeb[random_indices]  # Select the random subse

import random
# Randomly select 2500 distinct images
random_indices = random.sample(range(len(ffhq)), 2500)  # Get 2500 random indices
ffhq = ffhq[random_indices]  # Select the random subse
print(f"celeb shape: {celeb.shape}, dtype: {celeb.dtype}")
print(f"ffhq shape: {ffhq.shape}, dtype: {ffhq.dtype}")
print(f"gdwct shape: {gdwct.shape}, dtype: {gdwct.dtype}")
print(f"attagan shape: {attgan.shape}, dtype: {attgan.dtype}")
print(f"stargan shape: {stargan.shape}, dtype: {stargan.dtype}")
print(f"stylegan2 shape: {stylegan2.shape}, dtype: {stylegan2.dtype}")
print(f"stylegan shape: {stylegan.shape}, dtype: {stylegan.dtype}")
import random
import numpy as np

def split_data(data, train_ratio=0.7):
    """
    Splits data into training and testing sets based on the specified ratio.

    Parameters:
        data (list or np.array): The dataset to split.
        train_ratio (float): The ratio of the data to include in the training set.

    Returns:
        tuple: Two datasets - train and test.
    """
    # Shuffle the data
    random.shuffle(data)

    # Calculate the split index
    split_index = int(len(data) * train_ratio)

    # Split the data
    train_data = data[:split_index]
    test_data = data[split_index:]

    return train_data, test_data

# Split `celeb` into 70% train and 30% test
celeb_train_hog, celeb_test_hog = split_data(celeb, train_ratio=0.7)

# Split `ffhq` into 70% train and 30% test
ffhq_train_hog, ffhq_test_hog = split_data(ffhq, train_ratio=0.7)

# Split `attgan` into 70% train and 30% test
attgan_train_hog, attgan_test_hog = split_data(attgan, train_ratio=0.7)

# Split `stargan` into 70% train and 30% test
stargan_train_hog, stargan_test_hog = split_data(stargan, train_ratio=0.7)

# Split `gdwct` into 70% train and 30% test
gdwct_train_hog, gdwct_test_hog = split_data(gdwct, train_ratio=0.7)

# Split `stylegan2` into 70% train and 30% test_hog
stylegan2_train_hog, stylegan2_test_hog = split_data(stylegan2, train_ratio=0.7)

# Split `stylegan` into 70% train and 30% test_hog
stylegan_train_hog, stylegan_test_hog = split_data(stylegan, train_ratio=0.7)

# Convert to NumPy arrays if needed
celeb_train_hog, celeb_test_hog = np.array(celeb_train_hog), np.array(celeb_test_hog)
ffhq_train_hog, ffhq_test_hog = np.array(ffhq_train_hog), np.array(ffhq_test_hog)
attgan_train_hog, attgan_test_hog = np.array(attgan_train_hog), np.array(attgan_test_hog)
stargan_train_hog, stargan_test_hog = np.array(stargan_train_hog), np.array(stargan_test_hog)
gdwct_train_hog, gdwct_test_hog = np.array(gdwct_train_hog), np.array(gdwct_test_hog)
stylegan2_train_hog, stylegan2_test_hog = np.array(stylegan2_train_hog), np.array(stylegan2_test_hog)
stylegan_train_hog, stylegan_test_hog = np.array(stylegan_train_hog), np.array(stylegan_test_hog)

# Print results for verification
print(f"celeb_train: {len(celeb_train_hog)} images, celeb_test: {len(celeb_test_hog)} images")
print(f"ffhq_train: {len(ffhq_train_hog)} images, ffhq_test: {len(ffhq_test_hog)} images")
print(f"attgan_train: {len(attgan_train_hog)} images, attgan_test: {len(attgan_test_hog)} images")
print(f"stargan_train: {len(stargan_train_hog)} images, stargan_test: {len(stargan_test_hog)} images")
print(f"gdwct_train: {len(gdwct_train_hog)} images, gdwct_test: {len(gdwct_test_hog)} images")
print(f"stylegan2_train: {len(stylegan2_train_hog)} images, stylegan2_test: {len(stylegan2_test_hog)} images")
print(f"stylegan_train: {len(stylegan_train_hog)} images, stylegan_test: {len(stylegan_test_hog)} images")

########################################################################################################################################
#######################################divide into 60,10 train and val
#########################################################################################################################################
def extract_validation(train_data):
    """
    Extract every 10th sample from the training data and store it in a validation set.

    Parameters:
        train_data (list or np.array): The training dataset.

    Returns:
        tuple: Updated training dataset and validation dataset.
    """
    # Select every 10th sample for the validation set
    validation_data = train_data[::10]

    # Remove the selected samples from the training dataset
    updated_train_data = [train_data[i] for i in range(len(train_data)) if i % 10 != 0]

    return np.array(updated_train_data), np.array(validation_data)


# Perform the operation for each dataset
celeb_train_hog, celeb_val_hog = extract_validation(celeb_train_hog)
ffhq_train_hog, ffhq_val_hog = extract_validation(ffhq_train_hog)
attgan_train_hog, attgan_val_hog = extract_validation(attgan_train_hog)
stargan_train_hog, stargan_val_hog = extract_validation(stargan_train_hog)
gdwct_train_hog, gdwct_val_hog = extract_validation(gdwct_train_hog)
stylegan2_train_hog, stylegan2_val_hog = extract_validation(stylegan2_train_hog)
stylegan_train_hog, stylegan_val_hog = extract_validation(stylegan_train_hog)

# Print results for verification
print(f"celeb_train: {len(celeb_train_hog)} images, celeb_val: {len(celeb_val_hog)} images")
print(f"ffhq_train: {len(ffhq_train_hog)} images, ffhq_val: {len(ffhq_val_hog)} images")
print(f"attgan_train: {len(attgan_train_hog)} images, attgan_val: {len(attgan_val_hog)} images")
print(f"stargan_train: {len(stargan_train_hog)} images, stargan_val: {len(stargan_val_hog)} images")
print(f"gdwct_train: {len(gdwct_train_hog)} images, gdwct_val: {len(gdwct_val_hog)} images")
print(f"stylegan2_train: {len(stylegan2_train_hog)} images, stylegan2_val: {len(stylegan2_val_hog)} images")
print(f"stylegan_train: {len(stylegan_train_hog)} images, stylegan_val: {len(stylegan_val_hog)} images")
############################################################################################################################################################
#################################################concatenate the labels 0,1 real and fake
#############################################################################################################################################################


celeb_train_labels = np.zeros(len(celeb_train_hog), dtype=int)
ffhq_train_labels = np.zeros(len(ffhq_train_hog), dtype=int)
atta_train_labels = np.ones(len(attgan_train_hog), dtype=int)
star_train_labels = np.ones(len(stargan_train_hog), dtype=int)
gdwct_train_labels = np.ones(len(gdwct_train_hog), dtype=int)
stylegan2_train_labels = np.ones(len(stylegan2_train_hog), dtype=int)
stylegan_train_labels = np.ones(len(stylegan_train_hog), dtype=int)

# Concatenate all training datasets into a single `train` variable
train_hog = np.concatenate([celeb_train_hog, ffhq_train_hog, attgan_train_hog, stargan_train_hog, gdwct_train_hog, stylegan2_train_hog, stylegan_train_hog], axis=0)
train_labels=np.concatenate([celeb_train_labels, ffhq_train_labels, atta_train_labels, star_train_labels, gdwct_train_labels, stylegan2_train_labels,
                              stylegan_train_labels], axis=0)




celeb_test_labels = np.zeros(len(celeb_test_hog), dtype=int)
ffhq_test_labels = np.zeros(len(ffhq_test_hog), dtype=int)
atta_test_labels = np.ones(len(attgan_test_hog), dtype=int)
star_test_labels = np.ones(len(stargan_test_hog), dtype=int)
gdwct_test_labels = np.ones(len(gdwct_test_hog), dtype=int)
stylegan2_test_labels = np.ones(len(stylegan2_test_hog), dtype=int)
stylegan_test_labels = np.ones(len(stylegan_test_hog), dtype=int)

# Concatenate all testing datasets into a single `test` variable
test_hog = np.concatenate([celeb_test_hog, ffhq_test_hog, attgan_test_hog, stargan_test_hog, gdwct_test_hog, stylegan2_test_hog, stylegan_test_hog], axis=0)
test_labels = np.concatenate([celeb_test_labels, ffhq_test_labels, atta_test_labels, star_test_labels, gdwct_test_labels, stylegan2_test_labels,
                        stylegan_test_labels], axis=0)




celeb_val_labels = np.zeros(len(celeb_val_hog), dtype=int)
ffhq_val_labels = np.zeros(len(ffhq_val_hog), dtype=int)
atta_val_labels = np.ones(len(attgan_val_hog), dtype=int)
star_val_labels = np.ones(len(stargan_val_hog), dtype=int)
gdwct_val_labels = np.ones(len(gdwct_val_hog), dtype=int)
stylegan2_val_labels = np.ones(len(stylegan2_val_hog), dtype=int)
stylegan_val_labels = np.ones(len(stylegan_val_hog), dtype=int)

# Concatenate all validation datasets into a single `val` variable
val_hog = np.concatenate([celeb_val_hog, ffhq_val_hog, attgan_val_hog, stargan_val_hog, gdwct_val_hog, stylegan2_val_hog, stylegan_val_hog], axis=0)
val_labels = np.concatenate([celeb_val_labels, ffhq_val_labels, atta_val_labels, star_val_labels, gdwct_val_labels, stylegan2_val_labels,
                       stylegan_val_labels], axis=0)

# Print the results for verification
print(f"Total train: {len(train_hog)} images")
print(f"Total test: {len(test_hog)} images")
print(f"Total val: {len(val_hog)} images")


# Print results for verification
print(f"Train Labels: {len(train_labels)} ")
print(f"Test Labels: {len(test_labels)} ")
print(f"Val Labels: {len(val_labels)} ")



celeb shape: (5000, 224, 224, 3), dtype: uint8
ffhq shape: (5000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
celeb shape: (2500, 160, 160, 3), dtype: uint8
ffhq shape: (2500, 160, 160, 3), dtype: uint8
gdwct shape: (1000, 160, 160, 3), dtype: uint8
attagan shape: (1000, 160, 160, 3), dtype: uint8
stargan shape: (1000, 160, 160, 3), dtype: uint8
stylegan2 shape: (1000, 160, 160, 3), dtype: uint8
stylegan shape: (1000, 160, 160, 3), dtype: uint8
celeb_train: 1750 images, celeb_test: 750 images
ffhq_train: 1750 images, ffhq_test: 750 images
attgan_train: 700 images, attgan_test: 300 images
stargan_train: 700 images, stargan_test: 300 images
gdwct_train: 700 images, gdwct_test: 300 images
stylegan2_train: 700 images, stylegan2_test: 300 images
stylegan_train: 700 images, stylegan

In [5]:


# Train the model
history = model.fit(
    train_hog, train_labels,
    validation_data=(val_hog, val_labels),
    epochs=100,
    batch_size=32
)

Epoch 1/100
197/197 [==============================] - 23s 73ms/step - loss: 0.5248 - accuracy: 0.7297 - auc: 0.8124 - val_loss: 0.5132 - val_accuracy: 0.7586 - val_auc: 0.8592
Epoch 2/100
197/197 [==============================] - 13s 66ms/step - loss: 0.2456 - accuracy: 0.9002 - auc: 0.9634 - val_loss: 0.2999 - val_accuracy: 0.8714 - val_auc: 0.9519
Epoch 3/100
197/197 [==============================] - 13s 67ms/step - loss: 0.1440 - accuracy: 0.9478 - auc: 0.9866 - val_loss: 0.2707 - val_accuracy: 0.9057 - val_auc: 0.9609
Epoch 4/100
197/197 [==============================] - 22s 114ms/step - loss: 0.1038 - accuracy: 0.9621 - auc: 0.9927 - val_loss: 0.2392 - val_accuracy: 0.9200 - val_auc: 0.9737
Epoch 5/100
197/197 [==============================] - 20s 101ms/step - loss: 0.0736 - accuracy: 0.9733 - auc: 0.9960 - val_loss: 0.2106 - val_accuracy: 0.9486 - val_auc: 0.9814
Epoch 6/100
197/197 [==============================] - 33s 169ms/step - loss: 0.0581 - accuracy: 0.9798 - auc: 0.

In [7]:
# Evaluate on test set
test_loss, test_acc, test_auc = model.evaluate(test_hog, test_labels)
print(f'\nTest Accuracy of DFC on InceptionV3: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test_hog)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred_binary))

94/94 [==============================] - 3s 33ms/step - loss: 0.4074 - accuracy: 0.9260 - auc: 0.9672

Test Accuracy of DFC on InceptionV3: 92.60%
94/94 [==============================] - 4s 29ms/step
              precision    recall  f1-score   support

           0       0.92      0.93      0.93      1500
           1       0.93      0.92      0.93      1500

    accuracy                           0.93      3000
   macro avg       0.93      0.93      0.93      3000
weighted avg       0.93      0.93      0.93      3000



In [8]:
import tensorflow as tf
# After training the model
model.save('InceptionV3_160_dfc.h5')  # Saves the entire model to a file

In [9]:
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts


=== RESOURCE USAGE ===
CPU Usage: 9.7%
RAM Used: 1566.5 MB
Time Usage: 2751.7 s
GPU Memory Used: 494.0 MB
Power Consumption: 93W


In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('InceptionV3_160_dfc.h5')
test_loss, test_acc, test_auc = model.evaluate(test_hog, test_labels)
print(f'Test Accuracy of DFC on InceptionV3: {test_acc * 100:.2f}%')
# You can now use the model for testing or inference


94/94 [==============================] - 4s 32ms/step - loss: 0.4074 - accuracy: 0.9260 - auc: 0.9672
Test Accuracy of DFC on MInceptionV3: 92.60%


In [3]:
from tensorflow.keras.models import load_model
import tensorflow as tf
print("=== DATA LOADING ===")
start = monitor.get_stats()
# Load the saved model
model = tf.keras.models.load_model('InceptionV3_160_dfc.h5')
# Evaluate on test set
test_loss, test_acc,test_auc = model.evaluate(test_hog, test_labels)
print(f'Test Accuracy of Hog on InceptionV3: {test_acc * 100:.2f}%')
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts

=== DATA LOADING ===
94/94 [==============================] - 9s 58ms/step - loss: 0.4293 - accuracy: 0.9150 - auc: 0.9608
Test Accuracy of Hog on InceptionV3: 91.50%

=== RESOURCE USAGE ===
CPU Usage: 3.1%
RAM Used: 3401.4 MB
Time Usage: 15.8 s
GPU Memory Used: 506.8 MB
Power Consumption: 93W


In [12]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('InceptionV3_160_dfc.h5')
# Evaluate on test set
test_loss, test_acc, test_auc = model.evaluate(test, test_labels)
print(f'\nTest Accuracy of Celeb-df dataset on InceptionV3 for DFC: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred_binary))


340/340 [==============================] - 82s 238ms/step - loss: 2.4703 - accuracy: 0.5921 - auc: 0.5070

Test Accuracy of Celeb-df dataset on InceptionV3 for DFC: 59.21%
340/340 [==============================] - 89s 261ms/step
              precision    recall  f1-score   support

           0       0.27      0.29      0.28      2961
           1       0.73      0.70      0.72      7891

    accuracy                           0.59     10852
   macro avg       0.50      0.50      0.50     10852
weighted avg       0.60      0.59      0.60     10852



In [7]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('InceptionV3_160_dfc.h5')
# Evaluate on test set
test_loss, test_acc, test_auc = model.evaluate(test_ff, test_labels_ff)
print(f'\nTest Accuracy of FF++ on Inceptionnet from DFC: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test_ff)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels_ff, y_pred_binary))

169/169 [==============================] - 40s 231ms/step - loss: 2.8194 - accuracy: 0.5297 - auc: 0.4988

Test Accuracy of FF++ on Xceptionnet from DFC: 52.97%
169/169 [==============================] - 46s 265ms/step
              precision    recall  f1-score   support

           0       0.27      0.43      0.33      1468
           1       0.73      0.57      0.64      3911

    accuracy                           0.53      5379
   macro avg       0.50      0.50      0.49      5379
weighted avg       0.60      0.53      0.55      5379



# FF++

In [2]:
import h5py

# Open the HDF5 file for reading
with h5py.File("D://thesis//dataset//ff++//images_ff++_orignal_processed.h5", "r") as h5f:
    org_seq_actor_array = h5f["ori_actor"][:]
    org_seq_youtube_array = h5f["ori_youtube"][:]
print("Data loaded successfully from the HDF5 file.")
with h5py.File("D://thesis//dataset//ff++//images_ff++_fake1_processed.h5", "r") as h5f:
    meni_seq_dfd_array = h5f["mni_dfd"][:]
    meni_seq_df_array = h5f["mni_df"][:]
print("Data loaded successfully from the HDF5 file.")
with h5py.File("D://thesis//dataset//ff++//images_ff++_fake2_processed.h5", "r") as h5f:
    meni_seq_f2f_array = h5f["mni_f2f"][:]
    meni_seq_fshifter_array = h5f["mni_fshifter"][:]
print("Data loaded successfully from the HDF5 file.")
with h5py.File("D://thesis//dataset//ff++//images_ff++_fake3_processed.h5", "r") as h5f:
    meni_seq_fswap_array = h5f["mni_fswap"][:]
    meni_seq_nt_array = h5f["mni_nt"][:]
print("Data loaded successfully from the HDF5 file.")
# Output the shape of the resulting array
print(f"Shape of the concatenated array: {org_seq_actor_array.shape}")
print(f"Shape of the concatenated array: {org_seq_youtube_array.shape}")

print(f"Shape of the concatenated array: {meni_seq_dfd_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_df_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_fshifter_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_f2f_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_fswap_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_nt_array.shape}")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

ori_actor_train_real, ori_actor_test_real = train_test_split(org_seq_actor_array, test_size=0.3, random_state=42)
ori_youtube_train_real, ori_youtube_test_real = train_test_split(org_seq_youtube_array, test_size=0.3, random_state=42)
# Split the data into train (70%) and test (30%)
mni_dfd_train_fake, mni_dfd_test_fake = train_test_split(meni_seq_dfd_array, test_size=0.3, random_state=42)
mni_df_train_fake, mni_df_test_fake = train_test_split(meni_seq_df_array, test_size=0.3, random_state=42)
mni_f2f_train_fake, mni_f2f_test_fake = train_test_split(meni_seq_f2f_array, test_size=0.3, random_state=42)
mni_fshifter_train_fake, mni_fshifter_test_fake = train_test_split(meni_seq_fshifter_array, test_size=0.3, random_state=42)
mni_fswap_train_fake, mni_fswap_test_fake = train_test_split(meni_seq_fswap_array, test_size=0.3, random_state=42)
mni_nt_train_fake, mni_nt_test_fake = train_test_split(meni_seq_nt_array, test_size=0.3, random_state=42)


# Output the shape to confirm the split dimensions
print("Training data ACTOR real shape:", ori_actor_train_real.shape, "Testing real data shape:", ori_actor_test_real.shape)
print("Training data Youtube real shape:", ori_youtube_train_real.shape, "Testing real data shape:", ori_youtube_test_real.shape)

print("Training data DFD fake shape:", mni_dfd_train_fake.shape, "Testing fake data shape:", mni_dfd_test_fake.shape)
print("Training data DF fake shape:", mni_df_train_fake.shape, "Testing fake data shape:", mni_df_test_fake.shape)
print("Training data f2f fake shape:", mni_f2f_train_fake.shape, "Testing fake data shape:", mni_f2f_test_fake.shape)
print("Training data fshifter fake shape:", mni_fshifter_train_fake.shape, "Testing fake data shape:", mni_fshifter_test_fake.shape)
print("Training data fswap fake shape:", mni_fswap_train_fake.shape, "Testing fake data shape:", mni_fswap_test_fake.shape)
print("Training data nt fake shape:", mni_nt_train_fake.shape, "Testing fake data shape:", mni_nt_test_fake.shape)
########################################################################################################################################
#######################################divide into 60,10 train and val
#########################################################################################################################################
def extract_validation(train_data):
    """
    Extract every 10th sample from the training data and store it in a validation set.

    Parameters:
        train_data (list or np.array): The training dataset.

    Returns:
        tuple: Updated training dataset and validation dataset.
    """
    # Select every 10th sample for the validation set
    validation_data = train_data[::10]

    # Remove the selected samples from the training dataset
    updated_train_data = [train_data[i] for i in range(len(train_data)) if i % 10 != 0]

    return np.array(updated_train_data), np.array(validation_data)


# Perform the operation for each dataset
train_hog_ori_actor_real, val_hog_ori_actor_real = extract_validation(ori_actor_train_real)
train_hog_ori_youtube_real, val_hog_ori_youtube_real = extract_validation(ori_youtube_train_real)
train_hog_mni_dfd_fake, val_hog_mni_dfd_fake = extract_validation(mni_dfd_train_fake)
train_hog_mni_df_fake, val_hog_mni_df_fake = extract_validation(mni_df_train_fake)
train_hog_mni_f2f_fake, val_hog_mni_f2f_fake = extract_validation(mni_f2f_train_fake)
train_hog_mni_fshifter_fake, val_hog_mni_fshifter_fake = extract_validation(mni_fshifter_train_fake)
train_hog_mni_fswap_fake, val_hog_mni_fswap_fake = extract_validation(mni_fswap_train_fake)
train_hog_mni_nt_fake, val_hog_mni_nt_fake = extract_validation(mni_nt_train_fake)
# Print results for verification
print(f"train_ori_actor hog_real: {len(train_hog_ori_actor_real)} images, val_ori_actor hog_real: {len(val_hog_ori_actor_real)} images")
print(f"train_ ori_youtube hog_real: {len(train_hog_ori_youtube_real)} images, val_ ori_youtube hog_real: {len(val_hog_ori_youtube_real)} images")
print(f"train_hog_mni_dfd_fake: {len(train_hog_mni_dfd_fake)} images, val_hog_mni_dfd_fake: {len(val_hog_mni_dfd_fake)} images")
print(f"train_hog_mni_df_fake: {len(train_hog_mni_df_fake)} images, val_hog_mni_df_fake: {len(val_hog_mni_df_fake)} images")
print(f"train_hog_mni_f2f_fake: {len(train_hog_mni_f2f_fake)} images, val_hog_mni_f2f_fake: {len(val_hog_mni_f2f_fake)} images")
print(f"train_hog_mni_fshifter_fake: {len(train_hog_mni_fshifter_fake)} images, val_hog_mni_fshifter_fake: {len(val_hog_mni_fshifter_fake)} images")
print(f"train_hog_mni_fswap_fake: {len(train_hog_mni_fswap_fake)} images, val_hog_mni_fswap_fake: {len(val_hog_mni_fswap_fake)} images")
print(f"train_hog_mni_nt_fake: {len(train_hog_mni_nt_fake)} images, val_hog_mni_nt_fake: {len(val_hog_mni_nt_fake)} images")
############################################################################################################################################################
#################################################concatenate the labels 0,1 real and fake
#############################################################################################################################################################


train_ori_actor_labels_real = np.zeros(len(train_hog_ori_actor_real), dtype=int)
train_ori_youtube_labels_real = np.zeros(len(train_hog_ori_youtube_real), dtype=int)
train_mni_dfd_labels_fake = np.ones(len(train_hog_mni_dfd_fake), dtype=int)
train_mni_df_labels_fake = np.ones(len(train_hog_mni_df_fake), dtype=int)
train_mni_f2f_labels_fake = np.ones(len(train_hog_mni_f2f_fake), dtype=int)
train_mni_fshifter_labels_fake = np.ones(len(train_hog_mni_fshifter_fake), dtype=int)
train_mni_fswap_labels_fake = np.ones(len(train_hog_mni_fswap_fake), dtype=int)
train_mni_nt_labels_fake = np.ones(len(train_hog_mni_nt_fake), dtype=int)

test_ori_actor_labels_real = np.zeros(len(ori_actor_test_real), dtype=int)
test_ori_youtube_labels_real = np.zeros(len(ori_youtube_test_real), dtype=int)
test_mni_dfd_labels_fake = np.ones(len(mni_dfd_test_fake), dtype=int)
test_mni_df_labels_fake = np.ones(len(mni_df_test_fake), dtype=int)
test_mni_f2f_labels_fake = np.ones(len(mni_f2f_test_fake), dtype=int)
test_mni_fshifter_labels_fake = np.ones(len(mni_fshifter_test_fake), dtype=int)
test_mni_fswap_labels_fake = np.ones(len(mni_fswap_test_fake), dtype=int)
test_mni_nt_labels_fake = np.ones(len(mni_nt_test_fake), dtype=int)


val_ori_actor_labels_real = np.zeros(len(val_hog_ori_actor_real), dtype=int)
val_ori_youtube_labels_real = np.zeros(len(val_hog_ori_youtube_real), dtype=int)
val_mni_dfd_labels_fake = np.ones(len(val_hog_mni_dfd_fake), dtype=int)
val_mni_df_labels_fake = np.ones(len(val_hog_mni_df_fake), dtype=int)
val_mni_f2f_labels_fake = np.ones(len(val_hog_mni_f2f_fake), dtype=int)
val_mni_fshifter_labels_fake = np.ones(len(val_hog_mni_fshifter_fake), dtype=int)
val_mni_fswap_labels_fake = np.ones(len(val_hog_mni_fswap_fake), dtype=int)
val_mni_nt_labels_fake = np.ones(len(val_hog_mni_nt_fake), dtype=int)
##################################################################################################################

# Concatenate all training datasets into a single `train` variable
train_ff = np.concatenate([train_hog_ori_actor_real, train_hog_ori_youtube_real,train_hog_mni_dfd_fake,train_hog_mni_df_fake,
                            train_hog_mni_f2f_fake,train_hog_mni_fshifter_fake,train_hog_mni_fswap_fake,train_hog_mni_nt_fake], axis=0)

train_labels_ff = np.concatenate([train_ori_actor_labels_real, train_ori_youtube_labels_real,train_mni_dfd_labels_fake,train_mni_df_labels_fake,
                            train_mni_f2f_labels_fake,train_mni_fshifter_labels_fake,train_mni_fswap_labels_fake,train_mni_nt_labels_fake], axis=0)

# Concatenate all validation datasets into a single `val` variable
val_ff = np.concatenate([val_hog_ori_actor_real, val_hog_ori_youtube_real, val_hog_mni_dfd_fake, val_hog_mni_df_fake,
                            val_hog_mni_f2f_fake, val_hog_mni_fshifter_fake, val_hog_mni_fswap_fake, val_hog_mni_nt_fake], axis=0)
val_labels_ff = np.concatenate([val_ori_actor_labels_real, val_ori_youtube_labels_real, val_mni_dfd_labels_fake, val_mni_df_labels_fake,
                            val_mni_f2f_labels_fake, val_mni_fshifter_labels_fake, val_mni_fswap_labels_fake, val_mni_nt_labels_fake], axis=0)
# Concatenate all testing datasets into a single `test` variable
test_ff = np.concatenate([ori_actor_test_real, ori_youtube_test_real, mni_dfd_test_fake,
                           mni_df_test_fake, mni_f2f_test_fake, mni_fshifter_test_fake,
                           mni_fswap_test_fake, mni_nt_test_fake], axis=0)
test_labels_ff = np.concatenate([test_ori_actor_labels_real, test_ori_youtube_labels_real, test_mni_dfd_labels_fake, test_mni_df_labels_fake,
                            test_mni_f2f_labels_fake, test_mni_fshifter_labels_fake, test_mni_fswap_labels_fake, test_mni_nt_labels_fake], axis=0)


# Print the results for verification
# Print the results for verification
print(f"Total train: {len(train_ff)} images, and shape:{train_ff.shape}")
print(f"Total test: {len(test_ff)} images, and shape:{test_ff.shape}")
print(f"Total val: {len(val_ff)} images, and shape:{val_ff.shape}")


# Print results for verification
print(f"Train Labels: {len(train_labels_ff)} ")
print(f"Test Labels: {len(test_labels_ff)} ")
print(f"Val Labels: {len(val_labels_ff)} ")





Data loaded successfully from the HDF5 file.
Data loaded successfully from the HDF5 file.
Data loaded successfully from the HDF5 file.
Data loaded successfully from the HDF5 file.
Shape of the concatenated array: (2808, 160, 160, 3)
Shape of the concatenated array: (2083, 160, 160, 3)
Shape of the concatenated array: (3299, 160, 160, 3)
Shape of the concatenated array: (2083, 160, 160, 3)
Shape of the concatenated array: (2083, 160, 160, 3)
Shape of the concatenated array: (2083, 160, 160, 3)
Shape of the concatenated array: (1742, 160, 160, 3)
Shape of the concatenated array: (1742, 160, 160, 3)
Training data ACTOR real shape: (1965, 160, 160, 3) Testing real data shape: (843, 160, 160, 3)
Training data Youtube real shape: (1458, 160, 160, 3) Testing real data shape: (625, 160, 160, 3)
Training data DFD fake shape: (2309, 160, 160, 3) Testing fake data shape: (990, 160, 160, 3)
Training data DF fake shape: (1458, 160, 160, 3) Testing fake data shape: (625, 160, 160, 3)
Training data f

In [5]:
# Train the model
history = model.fit(
    train_ff, train_labels_ff,
    validation_data=(val_ff, val_labels_ff),
    epochs=100,
    batch_size=32
)

Epoch 1/100
353/353 [==============================] - 34s 67ms/step - loss: 0.5839 - accuracy: 0.7210 - auc: 0.6045 - val_loss: 0.5947 - val_accuracy: 0.7373 - val_auc: 0.6371
Epoch 2/100
353/353 [==============================] - 22s 62ms/step - loss: 0.5520 - accuracy: 0.7404 - auc: 0.6671 - val_loss: 0.5598 - val_accuracy: 0.7476 - val_auc: 0.6636
Epoch 3/100
353/353 [==============================] - 22s 63ms/step - loss: 0.5340 - accuracy: 0.7549 - auc: 0.6983 - val_loss: 0.6136 - val_accuracy: 0.7484 - val_auc: 0.6678
Epoch 4/100
353/353 [==============================] - 22s 62ms/step - loss: 0.5149 - accuracy: 0.7667 - auc: 0.7263 - val_loss: 0.5574 - val_accuracy: 0.7325 - val_auc: 0.6674
Epoch 5/100
353/353 [==============================] - 22s 62ms/step - loss: 0.4964 - accuracy: 0.7769 - auc: 0.7475 - val_loss: 0.5596 - val_accuracy: 0.7373 - val_auc: 0.6383
Epoch 6/100
353/353 [==============================] - 22s 62ms/step - loss: 0.4769 - accuracy: 0.7921 - auc: 0.772

In [7]:
# Evaluate on test set
test_loss, test_acc, test_auc = model.evaluate(test_ff, test_labels_ff)
print(f'\nTest Accuracy of FF++ on Inceptionnet: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test_ff)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels_ff, y_pred_binary))

169/169 [==============================] - 5s 32ms/step - loss: 1.5210 - accuracy: 0.7382 - auc: 0.6626

Test Accuracy of FF++ on Inceptionnet: 73.82%
169/169 [==============================] - 5s 30ms/step
              precision    recall  f1-score   support

           0       0.53      0.34      0.41      1468
           1       0.78      0.89      0.83      3911

    accuracy                           0.74      5379
   macro avg       0.66      0.61      0.62      5379
weighted avg       0.71      0.74      0.72      5379



In [8]:
import tensorflow as tf
# After training the model
model.save('Inceptionnet_160_ff.h5')  # Saves the entire model to a file

In [9]:
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts


=== RESOURCE USAGE ===
CPU Usage: 11.8%
RAM Used: 7973.4 MB
Time Usage: 2830.3 s
GPU Memory Used: 1980.9 MB
Power Consumption: 93W


In [3]:
from tensorflow.keras.models import load_model
import tensorflow as tf
print("=== DATA LOADING ===")
start = monitor.get_stats()
# Load the saved model
model = tf.keras.models.load_model('Inceptionnet_160_ff.h5')
# Evaluate on test set
test_loss, test_acc,test_auc = model.evaluate(test_ff, test_labels_ff)
print(f'Test Accuracy of ff++ on InceptionV3: {test_acc * 100:.2f}%')
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts

=== DATA LOADING ===
169/169 [==============================] - 12s 48ms/step - loss: 1.5210 - accuracy: 0.7381 - auc: 0.6626
Test Accuracy of ff++ on InceptionV3: 73.81%

=== RESOURCE USAGE ===
CPU Usage: 1.8%
RAM Used: 3414.1 MB
Time Usage: 18.7 s
GPU Memory Used: 681.0 MB
Power Consumption: 93W


In [18]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('Inceptionnet_160_ff.h5')
test_loss, test_acc, test_auc = model.evaluate(test_hog, test_labels)
print(f'Test Accuracy of dfc on Inceptionnet from FF++: {test_acc * 100:.2f}%')
# You can now use the model for testing or inference


94/94 [==============================] - 25s 250ms/step - loss: 2.7801 - accuracy: 0.4980 - auc: 0.5066
Test Accuracy of dfc on Inceptionnet from FF++: 49.80%


In [14]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('Inceptionnet_160_ff.h5')
# Evaluate on test set
test_loss, test_acc, test_auc = model.evaluate(test, test_labels)
print(f'\nTest Accuracy of Celeb-df dataset on Inceptionnet for FF++: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred_binary))


340/340 [==============================] - 207s 607ms/step - loss: 2.3547 - accuracy: 0.7264 - auc: 0.5482

Test Accuracy of Celeb-df dataset on Inceptionnet for FF++: 72.64%
340/340 [==============================] - 102s 298ms/step
              precision    recall  f1-score   support

           0       0.48      0.03      0.06      2961
           1       0.73      0.99      0.84      7891

    accuracy                           0.73     10852
   macro avg       0.60      0.51      0.45     10852
weighted avg       0.66      0.73      0.63     10852

